In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
#export
from pathlib import Path

### Concept Grounding
Extract and match concepts from Ai2Thor objects list to ConceptNet concepts.

In [ ]:
#export
from grounding.grounding_concepts import GroundConcepts

In [ ]:
#export
config_path = Path("/Users/odoemoo1/projects/visual-navigation/KagNet/grounding/paths.cfg")
ground = GroundConcepts(config_path)

In [ ]:
#export
ground.process('/Users/odoemoo1/projects/visual-navigation/KagNet/grounding/objects.txt', )

### Generate Conceptnet graph

Run below in cell:
```
!cd ./pathfinder/`
!python graph_construction.py`
!cd ..
```

Outputs networkX graph of conceptnet.

### Path finding
Find a path between the concepts (i.e. objects).

Re-factored pathfinder.py to create an object. 

In [ ]:
#export
from pathfinder.pathfinder import PathFinder

In [ ]:
#export
config_fn = '/Users/odoemoo1/projects/visual-navigation/KagNet/pathfinder/paths.cfg'
pathfinder = PathFinder(config_fn)
fn = Path('./datasets/ai2thor/ai2thor_objects_to_objects_rooms_include_self_concepts.json')

In [ ]:
#export
pathfinder.process(fn, nhops=1, beautify=True)

#### score paths
For concept and relation embeddings using glove embedding. In KagNet paper used knowledge graph embedding derived from TransE model. I will need to train this model to get the weights. For now use Glove embedding. 

In [ ]:
#export
from pathfinder.path_scoring import ScorePaths

In [ ]:
#export
scorer = ScorePaths(config_fn)
paths_fn = 'datasets/ai2thor/ai2thor_objects_to_objects_rooms_include_self_concepts_1hops_paths.json.pickle'
concepts_fn = 'datasets/ai2thor/ai2thor_objects_to_objects_rooms_include_self_concepts.json'
scorer.process(paths_fn, concepts_fn)

### Graph Generation
A graph is generated for each object in AI2-Thor, depicting relationships (based on ConceptNet knowledge graph)

In [ ]:
#export
from graph_generation.graph_gen import GenGraph

In [ ]:
#export
config_fn = '/Users/odoemoo1/projects/visual-navigation/KagNet/graph_generation/paths.cfg'
gen_graph = GenGraph(config_fn)

In [ ]:
#export
paths_fn = 'datasets/ai2thor/ai2thor_objects_to_objects_rooms_include_self_concepts_1hops_paths.json_scores_pruned.pickle'
concepts_fn = 'datasets/ai2thor/ai2thor_objects_to_objects_rooms_include_self_concepts.json'
gen_graph.process(concepts_fn, paths_fn)

### Visualize Graphs

In [ ]:
#export
nxgs = []
graph_ngx_file = 'datasets/ai2thor/ai2thor_objects_to_objects_rooms_include_self_concepts_1hops_paths.json_scores_pruned_graph'

with open(graph_ngx_file, 'r') as fr:
    for line in fr.readlines():
        line = line.strip()
        nxgs.append(line)

In [ ]:
from tqdm import tqdm
import networkx as nx, dgl, json, torch

In [ ]:
nxg = nx.node_link_graph(json.loads(nxgs[0]))
nxg.nodes[0]

In [ ]:
dgs = []
num_nodes, num_edges = [], []
for ii, nxg_str in tqdm(enumerate(nxgs), total=len(nxgs)):
    nxg = nx.node_link_graph(json.loads(nxg_str))
    dg = dgl.DGLGraph()
    dg.from_networkx(nxg)
    cids = [nxg.nodes[n_id]['cid'] + 1 for n_id in range(len(dg))] #adding 1, as 0 stands for placeholder concept
    dg.ndata.update({'conceptnet_ids': torch.LongTensor(cids)})
    num_nodes.append(dg.number_of_nodes())
    num_edges.append(dg.number_of_edges())
    dgs.append(dg)

In [ ]:
print(f'Num. unique edges: {np.unique(num_edges)} \t Num. unique nodes: {np.unique(num_nodes)}')

In [ ]:
nx.draw(nx.node_link_graph(json.loads(nxgs[-39])), with_labels=True)

In [ ]:
dgs[0].ndata['conceptnet_ids']

In [ ]:
dgs[-10].ndata['conceptnet_ids']

In [7]:
!python notebook2script.py --fname subgraph_generation.ipynb

Converted subgraph_generation.ipynb to subgraph_generation.py


In [4]:
%debug

ERROR:root:No traceback has been produced, nothing to debug.


## Git

In [ ]:
!git status

In [ ]:
!git add .gitignore

In [ ]:
!git commit -m "Refactor graph generation."